In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

trainset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Train", transform=transform_train)
testset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Test", transform=transform_test)

trainloader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)
class_names = trainset.classes

model = models.mobilenet_v2(pretrained=True)

for param in model.features.parameters():
    param.requires_grad = False

for name, param in model.features.named_parameters():
    if "17" in name or "18" in name:
        param.requires_grad = True

model.classifier[1] = nn.Linear(model.last_channel, len(class_names))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


model.train()
for epoch in range(30): 
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()
    print(f"Epoch {epoch+1}: Loss={running_loss:.3f}, Acc={100.*correct/total:.2f}%")

torch.save(model.state_dict(), "mobilenetv2_finetuned.pth")
print("\n💾 Model saved to mobilenetv2_finetuned.pth")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(testloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"\n✅ Test Accuracy: {100. * correct / total:.2f}%")


c:\Users\UJJWAL\OneDrive\Desktop\BirdClassification\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\UJJWAL\OneDrive\Desktop\BirdClassification\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\UJJWAL/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:03<00:00, 3.92MB/s]
Epoch 1: 100%|██████████| 589/589 [06:33<00:00,  1.50it/s]


Epoch 1: Loss=1992.808, Acc=23.04%


Epoch 2: 100%|██████████| 589/589 [05:48<00:00,  1.69it/s]


Epoch 2: Loss=1314.119, Acc=41.93%


Epoch 3: 100%|██████████| 589/589 [05:35<00:00,  1.76it/s]


...

Epoch 29: Loss=727.285, Acc=67.92%


Epoch 30: 100%|██████████| 589/589 [05:33<00:00,  1.77it/s]


Epoch 30: Loss=740.415, Acc=67.25%

💾 Model saved to mobilenetv2_finetuned.pth


100%|██████████| 149/149 [01:23<00:00,  1.78it/s]


✅ Test Accuracy: 73.88%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Train", transform=transform_train)
testset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Test", transform=transform_test)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_subset, val_subset = random_split(dataset, [train_size, val_size])

trainloader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=2)
valloader = DataLoader(val_subset, batch_size=16, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

class_names = dataset.classes

model = models.mobilenet_v2(pretrained=True)

for param in model.features.parameters():
    param.requires_grad = False

for name, param in model.features.named_parameters():
    if int(name.split('.')[0]) >= 14:
        param.requires_grad = True

model.classifier[1] = nn.Linear(model.last_channel, len(class_names))
model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=True)


best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(30):
    print(f"\n🔁 Epoch {epoch+1}")

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(trainloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss
    train_acc = 100. * correct / total
    print(f"📉 Train Loss: {train_loss:.3f} | ✅ Train Acc: {train_acc:.2f}%")
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(valloader, desc="Validating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100. * correct / total
    print(f"🧪 Val Loss: {val_loss:.3f} | 🎯 Val Acc: {val_acc:.2f}%")

    scheduler.step(val_loss)

    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, "mobilenetv2_best.pth")
        print("💾 Best model saved.")

model.load_state_dict(best_model_wts)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(testloader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"\n✅ Final Test Accuracy: {100. * correct / total:.2f}%")



c:\Users\UJJWAL\OneDrive\Desktop\BirdClassification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



🔁 Epoch 1


Training: 100%|██████████| 530/530 [07:51<00:00,  1.13it/s]


📉 Train Loss: 2492.200 | ✅ Train Acc: 5.67%


Validating: 100%|██████████| 59/59 [00:59<00:00,  1.00s/it]


...

📉 Train Loss: 993.288 | ✅ Train Acc: 74.80%


Validating: 100%|██████████| 59/59 [00:37<00:00,  1.57it/s]


🧪 Val Loss: 135.523 | 🎯 Val Acc: 62.00%


Testing: 100%|██████████| 149/149 [01:24<00:00,  1.76it/s]


✅ Final Test Accuracy: 77.46%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


dataset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Train", transform=transform_train)
testset = datasets.ImageFolder("C:/Users/UJJWAL/OneDrive/Desktop/BirdClassification/Test", transform=transform_test)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_subset, val_subset = random_split(dataset, [train_size, val_size])

trainloader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=2)
valloader = DataLoader(val_subset, batch_size=16, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

class_names = dataset.classes

from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
model.to(device)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(30):
    print(f"\n🔁 Epoch {epoch+1}")

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(trainloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss
    train_acc = 100. * correct / total
    print(f"📉 Train Loss: {train_loss:.3f} | ✅ Train Acc: {train_acc:.2f}%")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(valloader, desc="Validating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100. * correct / total
    print(f"🧪 Val Loss: {val_loss:.3f} | 🎯 Val Acc: {val_acc:.2f}%")

    scheduler.step()

    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, "efficientnetb0_best.pth")
        print("💾 Best model saved.")

model.load_state_dict(best_model_wts)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(testloader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"\n✅ Final Test Accuracy: {100. * correct / total:.2f}%")


c:\Users\UJJWAL\OneDrive\Desktop\BirdClassification\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\UJJWAL/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:04<00:00, 4.40MB/s]



🔁 Epoch 1


Training: 100%|██████████| 530/530 [20:26<00:00,  2.31s/it]


📉 Train Loss: 2508.138 | ✅ Train Acc: 10.39%


Validating: 100%|██████████| 59/59 [00:53<00:00,  1.10it/s]


...

📉 Train Loss: 837.695 | ✅ Train Acc: 83.20%


Validating: 100%|██████████| 59/59 [01:02<00:00,  1.06s/it]


🧪 Val Loss: 111.287 | 🎯 Val Acc: 71.55%


Testing: 100%|██████████| 149/149 [02:04<00:00,  1.20it/s]


✅ Final Test Accuracy: 84.12%
